In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/food-com-recipes-and-user-interactions/RAW_interactions.csv
/kaggle/input/food-com-recipes-and-user-interactions/ingr_map.pkl
/kaggle/input/food-com-recipes-and-user-interactions/PP_recipes.csv
/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv
/kaggle/input/food-com-recipes-and-user-interactions/interactions_train.csv
/kaggle/input/food-com-recipes-and-user-interactions/interactions_test.csv
/kaggle/input/food-com-recipes-and-user-interactions/PP_users.csv
/kaggle/input/food-com-recipes-and-user-interactions/interactions_validation.csv
/kaggle/input/gpt2-2/__results__.html
/kaggle/input/gpt2-2/__notebook__.ipynb
/kaggle/input/gpt2-2/__output__.json
/kaggle/input/gpt2-2/custom.css
/kaggle/input/gpt2-2/fine-tuned-gpt2/config.json
/kaggle/input/gpt2-2/fine-tuned-gpt2/merges.txt
/kaggle/input/gpt2-2/fine-tuned-gpt2/vocab.json
/kaggle/input/gpt2-2/fine-tuned-gpt2/tokenizer_config.json
/kaggle/input/gpt2-2/fine-tuned-gpt2/model.safetensors
/kaggle/input/gpt2-

In [2]:
# Read data
import pickle

ingr_map = pd.read_pickle("/kaggle/input/food-com-recipes-and-user-interactions/ingr_map.pkl")
input_path = "/kaggle/input/food-com-recipes-and-user-interactions/"
df_recipes = pd.read_csv(input_path + "RAW_recipes.csv")

print(df_recipes.shape)

(231637, 12)


In [3]:
df_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [4]:
import ast

replacement_dict = pd.Series(ingr_map['replaced'].values, index=ingr_map['raw_ingr']).to_dict()

# Function to convert and replace ingredients
def convert_and_replace(ingredient_str, replacement_dict):
    try:
        # Convert string representation of list to actual list
        ingredient_list = ast.literal_eval(ingredient_str)
        # Replace ingredients
        replaced_list = [replacement_dict.get(ingredient, ingredient) for ingredient in ingredient_list]
        return replaced_list
    except (ValueError, SyntaxError):
        # Handle cases where conversion fails
        return []

# Apply replacement to the ingredients column
df_recipes['ingredients'] = df_recipes['ingredients'].apply(lambda x: convert_and_replace(x, replacement_dict))

In [5]:
print(df_recipes.shape)
df_recipes.head()

(231637, 12)


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"[pizza crust, sausage, egg, milk, salt and pep...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onion, diced tomato, toma...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","[cheese, new potato, shallot, parsley, tarrago...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...",8


In [6]:
df_recipes = df_recipes.iloc[57909:115818]
print(df_recipes.shape)

(57909, 12)


In [7]:
# Combine the relevant columns into a new dataframe
df_simplified = df_recipes[['minutes', 'ingredients', 'steps']].copy()

# Format the ingredients as a comma-separated string for better readability
df_simplified['ingredients'] = df_simplified['ingredients'].apply(lambda x: ', '.join(x))

# Combine all columns into a single text block format for the transformer model
def format_recipe_simple(row):
    return (
        f"Cooking Time: {row['minutes']} minutes "
        f"Ingredients: {row['ingredients']} "
        f"Steps: {row['steps']} "
    )

df_simplified['formatted_recipe'] = df_simplified.apply(format_recipe_simple, axis=1)

df_simplified.head()

,minutes,ingredients,steps,formatted_recipe
57909,5,"pomegranate juice, orange juice, banana, straw...","['chop bananas into small pieces , easier to b...",Cooking Time: 5 minutes Ingredients: pomegrana...
57910,60,"chile, warm water, cilantro, garlic, fresh lim...","['soak chile in warm water until sotened , abo...","Cooking Time: 60 minutes Ingredients: chile, w..."
57911,25,"puffed corn cereal, butter, worcestershire sau...","['pour melted butter in a large mixing bowl', ...",Cooking Time: 25 minutes Ingredients: puffed c...
57912,50,"creamed corn, whole kernel corn, egg, sour cre...","['combine all ingredients in a big bowl', 'pou...",Cooking Time: 50 minutes Ingredients: creamed ...
57913,75,"corn kernel, fresh jalapeno pepper, garlic, re...","['defrost the corn , or , if fresh , boil for ...",Cooking Time: 75 minutes Ingredients: corn ker...


In [8]:
recipes = df_simplified['formatted_recipe'].tolist()

from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch

# Load the pre-trained GPT-2 model and tokenizer
# model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained('/kaggle/input/gpt2-2/fine-tuned-gpt2')
model = GPT2LMHeadModel.from_pretrained('/kaggle/input/gpt2-2/fine-tuned-gpt2')

# Set eos_token as pad_token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
inputs = tokenizer(recipes, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
dataset = torch.utils.data.TensorDataset(inputs["input_ids"], inputs["attention_mask"])

In [9]:
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

class RecipeDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Create the dataset
dataset = RecipeDataset(inputs)

In [10]:
# Define data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

import wandb

wandb.login(key='c65824cada15b84e31e7e9f35db6fe50eecfd857')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    warmup_steps=1000,
    learning_rate=5e-5,
    num_train_epochs=5,
    weight_decay=0.001,
    per_device_train_batch_size=4,  # Adjust based on your GPU's capacity
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
    fp16=True
)

# Use the Trainer class to fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("fine-tuned-gpt2")
tokenizer.save_pretrained("fine-tuned-gpt2")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: klmnopganesh (klmnopgan

Step,Training Loss
500,1.643000
1000,1.661600
1500,1.669700
2000,1.664000
2500,1.672400
3000,1.677800
3500,1.675500
4000,1.675600
4500,1.685300
5000,1.668300


('fine-tuned-gpt2/tokenizer_config.json',
 'fine-tuned-gpt2/special_tokens_map.json',
 'fine-tuned-gpt2/vocab.json',
 'fine-tuned-gpt2/merges.txt',
 'fine-tuned-gpt2/added_tokens.json')

In [11]:
import os

# Define the path
path = '/kaggle/working/'

# Walk through the directory and print files and subdirectories
for root, dirs, files in os.walk(path):
    print(f"Root directory: {root}")
    for directory in dirs:
        print(f"Subdirectory: {directory}")
    for file in files:
        print(f"File: {file}")

Root directory: /kaggle/working/
Subdirectory: results
Subdirectory: wandb
Subdirectory: fine-tuned-gpt2
File: __notebook__.ipynb
Root directory: /kaggle/working/results
Subdirectory: checkpoint-72000
Subdirectory: runs
Subdirectory: checkpoint-72390
Root directory: /kaggle/working/results/checkpoint-72000
File: config.json
File: training_args.bin
File: rng_state.pth
File: scheduler.pt
File: model.safetensors
File: optimizer.pt
File: trainer_state.json
File: generation_config.json
Root directory: /kaggle/working/results/runs
Subdirectory: Sep25_15-24-05_78ebbb3607d6
Root directory: /kaggle/working/results/runs/Sep25_15-24-05_78ebbb3607d6
File: events.out.tfevents.1727277850.78ebbb3607d6.23.0
Root directory: /kaggle/working/results/checkpoint-72390
File: config.json
File: training_args.bin
File: rng_state.pth
File: scheduler.pt
File: model.safetensors
File: optimizer.pt
File: trainer_state.json
File: generation_config.json
Root directory: /kaggle/working/wandb
Subdirectory: latest-run
S